In [3]:
import xarray as xr
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import requests 

# TAU X

In [7]:
file_url = "https://iridl.ldeo.columbia.edu/SOURCES/.CARTON-GIESE/.SODA/.v2p2p4/taux/data.nc"
	
r = requests.get(file_url, stream = True) 

with open("./taux.nc", "wb") as file: 
	for block in r.iter_content(chunk_size = 1024): 
		if block: 
			file.write(block) 
   
size = os.path.getsize('./taux.nc') 
print('Size of file is', size, 'bytes')

Size of file is 1573874016 bytes


In [60]:
ds_taux = xr.open_dataset("./taux.nc" , decode_times=False)
def fix_calendar(ds, timevar='time'):
    if ds[timevar].attrs['calendar'] == '360':
        ds[timevar].attrs['calendar'] = '360_day'
    return ds

ds_taux = fix_calendar(ds_taux)
ds_taux = xr.decode_cf(ds_taux)
ds_taux

<xarray.Dataset>
Dimensions:  (lat: 330, lon: 720, time: 1656)
Coordinates:
  * lat      (lat) float32 -75.25 -74.75 -74.25 -73.75 ... 88.25 88.75 89.25
  * lon      (lon) float32 0.25 0.75 1.25 1.75 2.25 ... 358.2 358.8 359.2 359.8
  * time     (time) object 1871-01-16 00:00:00 ... 2008-12-16 00:00:00
Data variables:
    taux     (time, lat, lon) float32 ...

In [61]:
ds_taux.where(((ds_taux.lon > 40 ) & (ds_taux.lon < 80 ) & (ds_taux.lat > 10) & (ds_taux.lat <20)), drop=True)

<xarray.Dataset>
Dimensions:  (lat: 20, lon: 80, time: 1656)
Coordinates:
  * lat      (lat) float32 10.25 10.75 11.25 11.75 ... 18.25 18.75 19.25 19.75
  * lon      (lon) float32 40.25 40.75 41.25 41.75 ... 78.25 78.75 79.25 79.75
  * time     (time) object 1871-01-16 00:00:00 ... 2008-12-16 00:00:00
Data variables:
    taux     (time, lat, lon) float32 nan nan nan nan nan ... nan nan nan nan

In [62]:
ds_taux = ds_taux.where(((ds_taux.lon > 40 ) & (ds_taux.lon < 80 ) & (ds_taux.lat > 10) & (ds_taux.lat <20)), drop=True)

In [63]:
df_taux= pd.DataFrame()
df_taux['time'] = np.array((ds_taux['time']))
df_taux['taux'] = ''
for i in range(1656):
  df_taux['taux'][i] = np.array((ds_taux['taux'][i]))
  

In [64]:
df_taux

,time,taux
0,1871-01-16 00:00:00,"[[nan, nan, nan, nan, nan, nan, nan, nan, -0.0..."
1,1871-02-16 00:00:00,"[[nan, nan, nan, nan, nan, nan, nan, nan, -0.0..."
2,1871-03-16 00:00:00,"[[nan, nan, nan, nan, nan, nan, nan, nan, -0.0..."
3,1871-04-16 00:00:00,"[[nan, nan, nan, nan, nan, nan, nan, nan, -0.0..."
4,1871-05-16 00:00:00,"[[nan, nan, nan, nan, nan, nan, nan, nan, 0.05..."
...,...,...
1651,2008-08-16 00:00:00,"[[nan, nan, nan, nan, nan, nan, nan, nan, 0.13..."
1652,2008-09-16 00:00:00,"[[nan, nan, nan, nan, nan, nan, nan, nan, 0.06..."
1653,2008-10-16 00:00:00,"[[nan, nan, nan, nan, nan, nan, nan, nan, 0.01..."
1654,2008-11-16 00:00:00,"[[nan, nan, nan, nan, nan, nan, nan, nan, -0.0..."


In [65]:
df_taux['Month'] = 0
for i in range(len(df_taux)):
  df_taux['Month'][i] =df_taux['time'][i].month

df_taux['Year'] = 0
for i in range(len(df_taux)):
  df_taux['Year'][i] =df_taux['time'][i].year

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [66]:
df_taux_april = df_taux.loc[df_taux['Month'] == 4]

In [67]:
df_taux_final = df_taux_april[['taux', 'Year']]

In [71]:
df_taux_final = df_taux_final.reset_index(drop=True)

In [72]:
for i in range(len(df_taux_final)):
  df_taux_final["taux"][i][np.isnan(df_taux_final["taux"][i])] = 0

In [73]:
df_taux_final

,taux,Year
0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0...",1871
1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0...",1872
2,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0...",1873
3,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0...",1874
4,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0...",1875
...,...,...
133,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0...",2004
134,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0...",2005
135,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0...",2006
136,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0...",2007


#SSH

In [6]:
file_url = "https://iridl.ldeo.columbia.edu/SOURCES/.CARTON-GIESE/.SODA/.v2p2p4/ssh/data.nc"
	
r = requests.get(file_url, stream = True) 

with open("./data.nc", "wb") as file: 
	for block in r.iter_content(chunk_size = 1024): 
		if block: 
			file.write(block) 

size = os.path.getsize('./data.nc') 
print('Size of file is', size, 'bytes')

Size of file is 1573874020 bytes


In [26]:
ds_ssh = xr.open_dataset("./data.nc" , decode_times=False)
def fix_calendar(ds, timevar='time'):
    if ds[timevar].attrs['calendar'] == '360':
        ds[timevar].attrs['calendar'] = '360_day'
    return ds

ds_ssh = fix_calendar(ds_ssh)
ds_ssh = xr.decode_cf(ds_ssh)
ds_ssh

<xarray.Dataset>
Dimensions:  (lat: 330, lon: 720, time: 1656)
Coordinates:
  * lon      (lon) float32 0.25 0.75 1.25 1.75 2.25 ... 358.2 358.8 359.2 359.8
  * lat      (lat) float32 -75.25 -74.75 -74.25 -73.75 ... 88.25 88.75 89.25
  * time     (time) object 1871-01-16 00:00:00 ... 2008-12-16 00:00:00
Data variables:
    ssh      (time, lat, lon) float32 ...

In [27]:
ds_ssh = ds_ssh.where(((ds_ssh.lon > 40 ) & (ds_ssh.lon < 80 ) & (ds_ssh.lat > 10) & (ds_ssh.lat <20)), drop=True)

In [28]:
ds_ssh['ssh']

<xarray.DataArray 'ssh' (time: 1656, lat: 20, lon: 80)>
array([[[       nan,        nan,        nan, ...,        nan,
         0.47368455, 0.4705949 ],
        [       nan,        nan,        nan, ...,        nan,
                nan, 0.47322145],
        [       nan,        nan,        nan, ...,        nan,
                nan, 0.45891967],
        ...,
        [0.12844115, 0.12032223, 0.14449105, ...,        nan,
                nan,        nan],
        [0.11885963, 0.10722669, 0.04978143, ...,        nan,
                nan,        nan],
        [0.09120873, 0.04488847,        nan, ...,        nan,
                nan,        nan]],

       [[       nan,        nan,        nan, ...,        nan,
         0.49529132, 0.49132225],
        [       nan,        nan,        nan, ...,        nan,
                nan, 0.4916404 ],
        [       nan,        nan,        nan, ...,        nan,
                nan, 0.47177067],
...
        [0.09632535, 0.09185884, 0.1074802 , ...,        nan,
                nan,        nan],
        [0.08791679, 0.09135956, 0.06385598, ...,        nan,
                nan,        nan],
        [0.08047532, 0.06369876,        nan, ...,        nan,
                nan,        nan]],

       [[       nan,        nan,        nan, ...,        nan,
         0.552477  , 0.5522664 ],
        [       nan,        nan,        nan, ...,        nan,
                nan, 0.5573779 ],
        [       nan,        nan,        nan, ...,        nan,
                nan, 0.54433674],
        ...,
        [0.11998665, 0.11525779, 0.13860157, ...,        nan,
                nan,        nan],
        [0.11312491, 0.11219507, 0.07148191, ...,        nan,
                nan,        nan],
        [0.09691926, 0.06839473,        nan, ...,        nan,
                nan,        nan]]], dtype=float32)
Coordinates:
  * lon      (lon) float32 40.25 40.75 41.25 41.75 ... 78.25 78.75 79.25 79.75
  * lat      (lat) float32 10.25 10.75 11.25 11.75 ... 18.25 18.75 19.25 19.75
  * time     (time) object 1871-01-16 00:00:00 ... 2008-12-16 00:00:00
Attributes:
    pointwidth:     1.0
    units:          meters
    standard_name:  sea_surface_height
    long_name:      SEA SURFACE HEIGHT

In [29]:
np.array(ds_ssh['ssh']).shape

(1656, 20, 80)

In [30]:
df_ssh= pd.DataFrame()
np.array((ds_ssh['time'][0]))
df_ssh['time'] = np.array((ds_ssh['time']))
df_ssh['ssh'] = ''
for i in range(1656):
  df_ssh['ssh'][i] = np.array((ds_ssh['ssh'][i]))
  

In [31]:
df_ssh

,time,ssh
0,1871-01-16 00:00:00,"[[nan, nan, nan, nan, nan, nan, nan, nan, 0.26..."
1,1871-02-16 00:00:00,"[[nan, nan, nan, nan, nan, nan, nan, nan, 0.29..."
2,1871-03-16 00:00:00,"[[nan, nan, nan, nan, nan, nan, nan, nan, 0.29..."
3,1871-04-16 00:00:00,"[[nan, nan, nan, nan, nan, nan, nan, nan, 0.27..."
4,1871-05-16 00:00:00,"[[nan, nan, nan, nan, nan, nan, nan, nan, 0.20..."
...,...,...
1651,2008-08-16 00:00:00,"[[nan, nan, nan, nan, nan, nan, nan, nan, -0.1..."
1652,2008-09-16 00:00:00,"[[nan, nan, nan, nan, nan, nan, nan, nan, -0.0..."
1653,2008-10-16 00:00:00,"[[nan, nan, nan, nan, nan, nan, nan, nan, 0.05..."
1654,2008-11-16 00:00:00,"[[nan, nan, nan, nan, nan, nan, nan, nan, 0.13..."


In [32]:
df_ssh['Month'] = 0
for i in range(len(df_ssh)):
  df_ssh['Month'][i] =df_ssh['time'][i].month

df_ssh['Year'] = 0
for i in range(len(df_ssh)):
  df_ssh['Year'][i] =df_ssh['time'][i].year

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [33]:
df_ssh_april = df_ssh.loc[df_ssh['Month'] == 4]

In [34]:
df_ssh_april

,time,ssh,Month,Year
3,1871-04-16 00:00:00,"[[nan, nan, nan, nan, nan, nan, nan, nan, 0.27...",4,1871
15,1872-04-16 00:00:00,"[[nan, nan, nan, nan, nan, nan, nan, nan, 0.24...",4,1872
27,1873-04-16 00:00:00,"[[nan, nan, nan, nan, nan, nan, nan, nan, 0.23...",4,1873
39,1874-04-16 00:00:00,"[[nan, nan, nan, nan, nan, nan, nan, nan, 0.22...",4,1874
51,1875-04-16 00:00:00,"[[nan, nan, nan, nan, nan, nan, nan, nan, 0.24...",4,1875
...,...,...,...,...
1599,2004-04-16 00:00:00,"[[nan, nan, nan, nan, nan, nan, nan, nan, 0.18...",4,2004
1611,2005-04-16 00:00:00,"[[nan, nan, nan, nan, nan, nan, nan, nan, 0.26...",4,2005
1623,2006-04-16 00:00:00,"[[nan, nan, nan, nan, nan, nan, nan, nan, 0.30...",4,2006
1635,2007-04-16 00:00:00,"[[nan, nan, nan, nan, nan, nan, nan, nan, 0.28...",4,2007


In [39]:
df_VK08_TCCount = pd.read_table('./VK08_TCCounts_1878-2020.dat', sep="\s+", header=None)
df_VK08_TCCount = df_VK08_TCCount.rename(columns={0: "Year", 1:"VK08_TCCount"}).copy()
df_VK08_TCCount

,Year,VK08_TCCount
0,1878,15.370
1,1879,11.308
2,1880,14.084
3,1881,10.234
4,1882,9.183
...,...,...
138,2016,15.000
139,2017,17.000
140,2018,15.000
141,2019,18.000


In [40]:
df_ssh_final = pd.merge(df_ssh_april, df_VK08_TCCount, on="Year")

In [41]:
df_ssh_final['Outlier'] = (df_ssh_final['VK08_TCCount'] > 14)

In [42]:
df_ssh_final['Outlier'].value_counts()

False    112
True      19
Name: Outlier, dtype: int64

In [43]:
df_ssh_final['Outlier'] = df_ssh_final['Outlier'].map({True:1, False:0})

In [44]:
df_ssh_final['ssh'][0].shape

(20, 80)

In [45]:
len(df_ssh_final["ssh"][0])

20

In [46]:
for i in range(len(df_ssh_final)):
  df_ssh_final["ssh"][i][np.isnan(df_ssh_final["ssh"][i])] = 0

In [47]:
df_ssh_final

,time,ssh,Month,Year,VK08_TCCount,Outlier
0,1878-04-16 00:00:00,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.24...",4,1878,15.370,1
1,1879-04-16 00:00:00,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.22...",4,1879,11.308,0
2,1880-04-16 00:00:00,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.19...",4,1880,14.084,1
3,1881-04-16 00:00:00,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.19...",4,1881,10.234,0
4,1882-04-16 00:00:00,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.21...",4,1882,9.183,0
...,...,...,...,...,...,...
126,2004-04-16 00:00:00,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.18...",4,2004,15.000,1
127,2005-04-16 00:00:00,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.26...",4,2005,28.000,1
128,2006-04-16 00:00:00,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.30...",4,2006,10.000,0
129,2007-04-16 00:00:00,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.28...",4,2007,15.000,1


#Combine DataFrames

In [74]:
df_final = pd.merge(df_ssh_final,df_taux_final, on="Year")

In [75]:
df_final['SuperFeature'] = ''
for i in range(len(df_final['SuperFeature'])):
  array = []
  array.append(df_final['ssh'][i])
  array.append(df_final['taux'][i])
  df_final['SuperFeature'][i] = array

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [76]:
df_final

,time,ssh,Month,Year,VK08_TCCount,Outlier,taux,SuperFeature
0,1878-04-16 00:00:00,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.24...",4,1878,15.370,1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2..."
1,1879-04-16 00:00:00,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.22...",4,1879,11.308,0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2..."
2,1880-04-16 00:00:00,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.19...",4,1880,14.084,1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1..."
3,1881-04-16 00:00:00,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.19...",4,1881,10.234,0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1..."
4,1882-04-16 00:00:00,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.21...",4,1882,9.183,0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2..."
...,...,...,...,...,...,...,...,...
126,2004-04-16 00:00:00,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.18...",4,2004,15.000,1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1..."
127,2005-04-16 00:00:00,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.26...",4,2005,28.000,1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2..."
128,2006-04-16 00:00:00,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.30...",4,2006,10.000,0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3..."
129,2007-04-16 00:00:00,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.28...",4,2007,15.000,1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2..."


In [86]:
df_final["SuperFeature"] = np.asarray(df_final["SuperFeature"])
for i in range(len(df_final["SuperFeature"])):
  df_final["SuperFeature"][i] = np.asarray(df_final["SuperFeature"][i])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


#SSH Convolutional NN 

In [ ]:
for i in  range(len(df_ssh_final['ssh'])):
   #df_ssh_final['ssh'][i] = np.array(df_ssh_final['ssh'][i])
   array = []
   df_ssh_final['ssh'][i] = np.asarray(df_ssh_final['ssh'][i].reshape(20,80,1))
   for x in range(len(df_ssh_final['ssh'][i])):
     df_ssh_final['ssh'][i][x] = np.asarray(df_ssh_final['ssh'][i][x])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
df_ssh_final['ssh'][0].reshape(20,80,1)

array([[[0.        ],
        [0.        ],
        [0.        ],
        ...,
        [0.        ],
        [0.39966956],
        [0.4007966 ]],

       [[0.        ],
        [0.        ],
        [0.        ],
        ...,
        [0.        ],
        [0.        ],
        [0.40092903]],

       [[0.        ],
        [0.        ],
        [0.        ],
        ...,
        [0.        ],
        [0.        ],
        [0.4039477 ]],

       ...,

       [[0.13599914],
        [0.12159704],
        [0.1391036 ],
        ...,
        [0.        ],
        [0.        ],
        [0.        ]],

       [[0.1184151 ],
        [0.11771224],
        [0.08147631],
        ...,
        [0.        ],
        [0.        ],
        [0.        ]],

       [[0.10600599],
        [0.08322307],
        [0.        ],
        ...,
        [0.        ],
        [0.        ],
        [0.        ]]], dtype=float32)

In [ ]:
df_ssh_final

,time,ssh,Month,Year,VK08_TCCount,Outlier
0,1878-04-16 00:00:00,"[[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...",4,1878,15.370,1
1,1879-04-16 00:00:00,"[[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...",4,1879,11.308,0
2,1880-04-16 00:00:00,"[[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...",4,1880,14.084,1
3,1881-04-16 00:00:00,"[[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...",4,1881,10.234,0
4,1882-04-16 00:00:00,"[[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...",4,1882,9.183,0
...,...,...,...,...,...,...
126,2004-04-16 00:00:00,"[[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...",4,2004,15.000,1
127,2005-04-16 00:00:00,"[[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...",4,2005,28.000,1
128,2006-04-16 00:00:00,"[[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...",4,2006,10.000,0
129,2007-04-16 00:00:00,"[[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...",4,2007,15.000,1


In [ ]:
X =  df_ssh_final['ssh'].to_list()
y =  df_ssh_final['Outlier']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=42)

In [ ]:
X_train =  np.asarray(X_train)
X_test =  np.asarray(X_test)
y_train =  np.asarray(y_train)
y_test =  np.asarray(y_test)

In [ ]:
from tensorflow.keras import layers, Model, Sequential

model = Sequential()
model.add(layers.Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(20,80,1)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.25))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2, activation='softmax'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 18, 78, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 76, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 8, 38, 64)        0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 8, 38, 64)         0         
                                                                 
 flatten (Flatten)           (None, 19456)             0         
                                                                 
 dense (Dense)               (None, 128)               2490496   
                                                        

In [ ]:
import tensorflow as tf

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=10, 
                    validation_data=(X_test, y_test))

Epoch 1/10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


3/3 [==============================] - 1s 261ms/step - loss: 0.6544 - accuracy: 0.6813 - val_loss: 0.3770 - val_accuracy: 0.8750
Epoch 2/10
3/3 [==============================] - 1s 185ms/step - loss: 0.4614 - accuracy: 0.8462 - val_loss: 0.4170 - val_accuracy: 0.8750
Epoch 3/10
3/3 [==============================] - 1s 174ms/step - loss: 0.4536 - accuracy: 0.8462 - val_loss: 0.3890 - val_accuracy: 0.8750
Epoch 4/10
3/3 [==============================] - 1s 183ms/step - loss: 0.4204 - accuracy: 0.8462 - val_loss: 0.3780 - val_accuracy: 0.8750
Epoch 5/10
3/3 [==============================] - 1s 180ms/step - loss: 0.4662 - accuracy: 0.8462 - val_loss: 0.3784 - val_accuracy: 0.8750
Epoch 6/10
3/3 [==============================] - 1s 188ms/step - loss: 0.4776 - accuracy: 0.8462 - val_loss: 0.4132 - val_accuracy: 0.8750
Epoch 7/10
3/3 [==============================] - 1s 174ms/step - loss: 0.4617 - accuracy: 0.8462 - val_loss: 0.3929 - val_accuracy: 0.8750
Epoch 8/10
3/3 [===============

In [ ]:
model.predict(X_test)

array([[0.7887295 , 0.21127059],
       [0.7487187 , 0.2512813 ],
       [0.7477472 , 0.2522528 ],
       [0.7537629 , 0.24623708],
       [0.7897716 , 0.21022843],
       [0.78096265, 0.21903737],
       [0.7510176 , 0.24898246],
       [0.8056134 , 0.19438659],
       [0.81386757, 0.18613242],
       [0.76855195, 0.23144801],
       [0.7890646 , 0.21093541],
       [0.7476202 , 0.25237972],
       [0.80666286, 0.19333713],
       [0.7522887 , 0.24771129],
       [0.78694403, 0.21305591],
       [0.8029117 , 0.19708826],
       [0.7542527 , 0.24574733],
       [0.8005242 , 0.19947578],
       [0.79285365, 0.20714633],
       [0.80078346, 0.19921654],
       [0.73864037, 0.26135963],
       [0.7588918 , 0.24110822],
       [0.79233825, 0.20766176],
       [0.7727622 , 0.22723782],
       [0.7871769 , 0.21282311],
       [0.76721877, 0.23278123],
       [0.7783258 , 0.22167416],
       [0.7964901 , 0.20350994],
       [0.7672283 , 0.23277172],
       [0.763193  , 0.23680696],
       [0.

In [ ]:
y_test

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0])

#Multie Feature 3D CNN

In [87]:
for i in  range(len(df_final)):
   array = []
   df_final['SuperFeature'][i] = np.asarray(df_final['SuperFeature'][i].reshape(2,20,80,1))
   for x in range(len(df_final['SuperFeature'][i])):
     df_final['SuperFeature'][i][x] = np.asarray(df_final['SuperFeature'][i][x])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [89]:
df_final

,time,ssh,Month,Year,VK08_TCCount,Outlier,taux,SuperFeature
0,1878-04-16 00:00:00,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.24...",4,1878,15.370,1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0...","[[[[0.], [0.], [0.], [0.], [0.], [0.], [0.], [..."
1,1879-04-16 00:00:00,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.22...",4,1879,11.308,0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00...","[[[[0.], [0.], [0.], [0.], [0.], [0.], [0.], [..."
2,1880-04-16 00:00:00,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.19...",4,1880,14.084,1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00...","[[[[0.], [0.], [0.], [0.], [0.], [0.], [0.], [..."
3,1881-04-16 00:00:00,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.19...",4,1881,10.234,0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01...","[[[[0.], [0.], [0.], [0.], [0.], [0.], [0.], [..."
4,1882-04-16 00:00:00,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.21...",4,1882,9.183,0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01...","[[[[0.], [0.], [0.], [0.], [0.], [0.], [0.], [..."
...,...,...,...,...,...,...,...,...
126,2004-04-16 00:00:00,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.18...",4,2004,15.000,1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0...","[[[[0.], [0.], [0.], [0.], [0.], [0.], [0.], [..."
127,2005-04-16 00:00:00,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.26...",4,2005,28.000,1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0...","[[[[0.], [0.], [0.], [0.], [0.], [0.], [0.], [..."
128,2006-04-16 00:00:00,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.30...",4,2006,10.000,0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0...","[[[[0.], [0.], [0.], [0.], [0.], [0.], [0.], [..."
129,2007-04-16 00:00:00,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.28...",4,2007,15.000,1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0...","[[[[0.], [0.], [0.], [0.], [0.], [0.], [0.], [..."


In [90]:
X =  df_final['SuperFeature'].to_list()
y =  df_final['Outlier']

In [91]:
from sklearn.model_selection import train_test_split

In [92]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=42)

In [93]:
X_train =  np.asarray(X_train)
X_test =  np.asarray(X_test)
y_train =  np.asarray(y_train)
y_test =  np.asarray(y_test)

In [109]:
from tensorflow.keras import layers, Model, Sequential

model = Sequential()
model.add(layers.Conv3D(32, kernel_size=(2,5,5),
                 activation='relu',
                 input_shape=(2,20,80,1)))
model.add(layers.Conv3D(64, (1,1, 1), activation='relu'))
model.add(layers.MaxPooling3D(pool_size=(1, 2,2)))
model.add(layers.Dropout(0.25))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2, activation='softmax'))

In [110]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_13 (Conv3D)          (None, 1, 16, 76, 32)     1632      
                                                                 
 conv3d_14 (Conv3D)          (None, 1, 16, 76, 64)     2112      
                                                                 
 max_pooling3d_3 (MaxPooling  (None, 1, 8, 38, 64)     0         
 3D)                                                             
                                                                 
 dropout_2 (Dropout)         (None, 1, 8, 38, 64)      0         
                                                                 
 flatten_1 (Flatten)         (None, 19456)             0         
                                                                 
 dense_2 (Dense)             (None, 128)               2490496   
                                                      

In [111]:
import tensorflow as tf

In [112]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [113]:
history = model.fit(X_train, y_train, epochs=10, 
                    validation_data=(X_test, y_test))

Epoch 1/10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


3/3 [==============================] - 2s 601ms/step - loss: 0.4863 - accuracy: 0.7363 - val_loss: 0.4755 - val_accuracy: 0.8750
Epoch 2/10
3/3 [==============================] - 2s 677ms/step - loss: 0.5336 - accuracy: 0.8462 - val_loss: 0.3793 - val_accuracy: 0.8750
Epoch 3/10
3/3 [==============================] - 2s 648ms/step - loss: 0.4268 - accuracy: 0.8462 - val_loss: 0.4197 - val_accuracy: 0.8750
Epoch 4/10
3/3 [==============================] - 2s 590ms/step - loss: 0.4773 - accuracy: 0.8462 - val_loss: 0.4188 - val_accuracy: 0.8750
Epoch 5/10
3/3 [==============================] - 2s 660ms/step - loss: 0.4788 - accuracy: 0.8462 - val_loss: 0.3804 - val_accuracy: 0.8750
Epoch 6/10
3/3 [==============================] - 1s 409ms/step - loss: 0.4336 - accuracy: 0.8462 - val_loss: 0.3773 - val_accuracy: 0.8750
Epoch 7/10
3/3 [==============================] - 1s 419ms/step - loss: 0.4324 - accuracy: 0.8462 - val_loss: 0.3770 - val_accuracy: 0.8750
Epoch 8/10
3/3 [===============

In [107]:
model.predict(X_test)

array([[0.83802414, 0.1619759 ],
       [0.8093459 , 0.19065408],
       [0.7850958 , 0.21490419],
       [0.80673087, 0.1932691 ],
       [0.8452519 , 0.15474811],
       [0.83002454, 0.16997543],
       [0.79892874, 0.20107126],
       [0.8632907 , 0.13670923],
       [0.8676149 , 0.13238509],
       [0.8065906 , 0.19340938],
       [0.84119105, 0.15880893],
       [0.7940605 , 0.20593949],
       [0.8609838 , 0.13901617],
       [0.7776217 , 0.22237836],
       [0.84567183, 0.15432817],
       [0.8570053 , 0.14299473],
       [0.80926937, 0.19073066],
       [0.85770965, 0.14229031],
       [0.84319174, 0.15680818],
       [0.8553963 , 0.14460367],
       [0.77499515, 0.22500484],
       [0.79486036, 0.20513965],
       [0.83972275, 0.16027719],
       [0.826184  , 0.17381601],
       [0.843706  , 0.15629394],
       [0.8225538 , 0.17744611],
       [0.8339179 , 0.1660821 ],
       [0.85563564, 0.1443644 ],
       [0.8190816 , 0.18091843],
       [0.8196745 , 0.18032551],
       [0.

In [108]:
y_test

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0])